In [9]:
import numpy as np
import pandas as pd
import redis
import pickle
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp

seen_car_ids = None
seen_car_ids = seen_car_ids and list(seen_car_ids)
__always_supported = ('md5', 'sha1', 'sha224', 'sha256', 'sha384', 'sha512')

class ModelData:
    user_features = 'user_features'
    item_features = 'item_features'
    trained_model = 'model'
    popular_listings = 'popular_listings'
    max_buyer_id = 'max_buyer_id'
    active_listings = 'active_listings'

df=pd.read_csv('/home/pavank/Desktop/mumbai.csv')


In [10]:
TOP_ORDER_LISTINGS_STATUSES = ['active', 'refurbishing']

def pandas_cached_query(query, disable_cache=False):
    if disable_cache:
        return pd.read_sql_query(query, connection)
    key = sha1(query).hexdigest()
    if key in cache:
        return cache.get(key)
    else:
        data = pd.read_sql_query(query, connection)
        cache.set(key, data, timeout=Settings.CACHALOT_TIMEOUT)
        return data
    
LISTING_FEATURE_QUERY = """SELECT listings.id, localities.city_id, IFNULL(price, 0) < 200000 AS price_b_2,
                           IFNULL(price, 0) BETWEEN 100000 AND 300000 AS price_bw_1_3, IFNULL(price, 0) BETWEEN 200000
                           AND 400000 AS price_bw_2_4, IFNULL(price, 0) BETWEEN 300000 AND 500000 AS price_bw_3_5,
                           IFNULL(price, 0) BETWEEN 400000 AND 600000 AS price_bw_4_6, IFNULL(price, 0) BETWEEN 500000
                           AND 700000 AS price_bw_5_7, IFNULL(price, 0) BETWEEN 600000 AND 800000 AS price_bw_6_8,
                           IFNULL(price, 0) BETWEEN 700000 AND 900000 AS price_bw_7_9, IFNULL(price, 0) > 900000 AS
                           price_a_9, IFNULL(listings.body_type_id = 1, 0) AS hatchback, IFNULL(listings.body_type_id = 
                           2, 0) AS sedan, IFNULL(listings.body_type_id = 3, 0) AS mpv, IFNULL(listings.body_type_id = 
                           4, 0) AS suv, listings.fuel_type_id_primary = 1 AS petrol, listings.fuel_type_id_primary = 2 
                           AS diesel, IFNULL(owners, 0) = 1 AS first_owner, IFNULL(owners, 0) > 1 AS owner, 
                           IFNULL(mileage, 0) <= 30000 AS mileage_b_3, IFNULL(mileage, 0) BETWEEN 30001 AND 70000 AS 
                           mileage_bw_3_7, IFNULL(mileage, 0) > 70000 AS mileage_a_7, IFNULL(year(curdate())-
                           year(registration_date), 0) <= 2 AS age_b_2, IFNULL(year(curdate())-year(registration_date), 
                           0) BETWEEN 3 AND 4 AS age_bw_3_4, IFNULL (year(curdate())- year(registration_date), 0) 
                           BETWEEN 5 AND 7 AS age_bw_5_7, IFNULL(year(curdate())- year(registration_date), 0)>7 AS 
                           age_a_7,  IFNULL(variants.model_id, 0) IN (SELECT variants.model_id FROM listings JOIN 
                           variants ON listings.variant_id=variants.id WHERE listings.id IN ({listing_ids})) AS model_id,
                           IFNULL(variants.id, 0) IN (SELECT variants.id FROM listings JOIN variants ON listings.
                           variant_id=variants.id WHERE listings.id IN ({listing_ids})) AS variant_id, IFNULL(listings.
                           color_id, 0) IN (1 , 3, 6) AS color_wsg, IFNULL(listings.color_id, 0) IN (2 , 13) AS color_bo,
                           IFNULL(listings.color_id, 0) NOT IN (1 , 2, 3, 6, 13) AS color_rest, IFNULL(CAST(listings.
                           is_inventory AS UNSIGNED), 0) AS is_inventory, price, created_at FROM listings JOIN 
                           localities ON listings.locality_id = localities.id JOIN variants ON listings.variant_id = 
                           variants.id where {filter_query}"""

In [11]:
pandas_cached_query(LISTING_FEATURE_QUERY)

NameError: global name 'sha1' is not defined

In [3]:
def get_trained_model(number):
    pool = redis.ConnectionPool(host="127.0.0.1", port=6379, db=1)
    redis_connection = redis.Redis(connection_pool=pool)
    ModelData.trained_model = pickle.loads(redis_connection.get('model'))
    ModelData.user_features = pickle.loads(redis_connection.get('user_features'))
    ModelData.item_features = pickle.loads(redis_connection.get('item_features'))
    ModelData.max_buyer_id = redis_connection.get('max_buyer_id')
    ModelData.popular_listings = pickle.loads(redis_connection.get('popular_listings'))
    ModelData.active_listings = pickle.loads(redis_connection.get('active_listings'))
    ModelData.pd_lt_2_set = pickle.loads(redis_connection.get('pd_lt_2_set'))
    ModelData.make_model_dict = ast.literal_eval(redis_connection.get('make_model_dict'))
    ModelData.features_dict = ast.literal_eval(redis_connection.get('features_dict'))
    ModelData.make_model_spell.word_frequency.load_words(ast.literal_eval(
        redis_connection.get('make_model_words')))
    ModelData.features_spell.word_frequency.load_words(
        ast.literal_eval(redis_connection.get('features_words')))

In [36]:
get_trained_model(1)

TypeError: StringIO() argument 1 must be string or buffer, not None

In [6]:
def get_listing_features(city_id, listings=None, inactive=False, listing_filter=True):
    city_filter = """localities.city_id = {city_id}""".format(city_id=city_id)
    listing_id_filter = """listings.id in ({listings})""".format(
        listings=','.join(map(str, list(listings)))
    ) if listings and listing_filter else None
    listing_status_filter = """listings.status in {status}""".format(
        status=tuple(TOP_ORDER_LISTINGS_STATUSES)
    ) if not inactive else None

    filter_array = [city_filter, listing_id_filter, listing_status_filter]

    listings_features = pandas_cached_query(
        LISTING_FEATURE_QUERY.format(
            filter_query=' and '.join(filter(None, filter_array)),
            listing_ids=','.join(map(str, listings)) if listings else 0
        )
    )

In [38]:
# def get_filtered__indices(features, filtered__ids):
#     features_mtx = features.tocoo()
#     filtered_indices = []
#     for item_id, feature_id in itertools.izip(features_mtx.row, features_mtx.col):
#         if feature_id in filtered__ids:
#             filtered_indices.append(item_id)
#     return filtered_indices

In [39]:
def get_item_representations(features=None):
    if features is None:
        return ModelData.trained_model.item_biases, ModelData.trained_model.item_embeddings

    features = sp.csr_matrix(features, dtype=np.float32)

    return features * ModelData.trained_model.item_biases, features * ModelData.trained_model.item_embeddings

In [40]:
# def get_similar_cars():
#     listings = [] 
#     city_active_listings_ids = df['2743'].tolist()
#     for id in city_active_listings_ids:
#         seen_car_ids = id
#         latest_seen_car_ids = seen_car_ids
#         features = ModelData.item_features
#         item_embeddings = get_item_representations(features)[1]
#         #data = ModelData.trained_model.mapping()[3]

# #         filtered_item_ids = [data[id] for id in latest_seen_car_ids]
# #         filtered_item_indices = get_filtered__indices(features, filtered_item_ids)
#         seen_item_embeddings = [item_embeddings[latest_seen_car_ids - 1]]

# #         city_active_listings = get_listing_features(1, listings=latest_seen_car_ids,listing_filter=False)
# #         city_active_listings_ids = np.array(city_active_listings)[:, 0]
# #         filtered_city_ids = [data[id] for id in city_active_listings_ids]
# #         filtered_city_indices = get_filtered__indices(features, filtered_city_ids)
#         city_active_listings_embeddings = []
#         for i in city_active_listings_ids:
#             city_active_listings_embeddings.append(item_embeddings[i - 1])

#         max_cosine_scores = -np.ones(len(city_active_listings)).reshape(1, -1)
#         for item_embeddings in seen_item_embeddings:
#             curr_cosine_scores = cosine_similarity(city_active_listings_embeddings, [item_embeddings]).reshape(1, -1)
#             max_cosine_scores = np.maximum(curr_cosine_scores, max_cosine_scores)

#         similarity_indices = np.argsort(-max_cosine_scores)
#         similar_cars_cosine_scores = max_cosine_scores

#         similar_listings = city_active_listings_ids[similarity_indices].ravel()
#         listings.append(similar_listings[:4])
    
#     f = pd.DataFrame(listings)
#     f.columns = ["Listing ID", "Reco - 1", "Reco - 2", "Reco - 3"]
#     f.to_csv('kkk.csv', encoding='utf-8', index=False)
#     return f

In [7]:
def get_similar_cars():
    listings = [] 
    coo = []
    get_item = []
    city_active_listings_ids = df['id'].tolist()
    for id in city_active_listings_ids:
        seen_car_ids = id
        latest_seen_car_ids = seen_car_ids
        listings_features = ModelData.item_features
        listings_item_embeddings = (listings_features.tocoo()) * ModelData.trained_model.item_embeddings
        coo.append(listings_item_embeddings)
        embeddings = get_item_representations(listings_features)[1]
        get_item.append(embeddings)
        seen_item_embeddings = [listings_item_embeddings[latest_seen_car_ids - 1]]
        city_active_listings_embeddings = []
        for i in city_active_listings_ids:
            city_active_listings_embeddings.append(listings_item_embeddings[i - 1])
        max_cosine_scores = -np.ones(len(city_active_listings_ids)).reshape(1, -1)
        for item_embeddings in seen_item_embeddings:
            curr_cosine_scores = cosine_similarity(city_active_listings_embeddings, [item_embeddings]).reshape(1, -1)
            max_cosine_scores = np.maximum(curr_cosine_scores, max_cosine_scores)
        similarity_indices = np.argsort(-max_cosine_scores)
        similar_cars_cosine_scores = max_cosine_scores
        city_active_listings_ids = np.array(city_active_listings_ids)
        similar_listings = city_active_listings_ids[similarity_indices].ravel()
        listings.append(similar_listings[:4])
#     f = pd.DataFrame(listings)
#     f.columns = ["Listing ID", "Reco - 1", "Reco - 2", "Reco - 3"]
#     f.to_csv('reco_test.csv', encoding='utf-8', index=False)
    return coo, get_item
        

In [8]:
get_similar_cars()

AttributeError: 'str' object has no attribute 'tocoo'

In [ ]:
def get_similar_cars():
    listings = [] 
    city_active_listings_ids = df['2743'].tolist()
    for id in city_active_listings_ids:
        seen_car_ids = id
        latest_seen_car_ids = seen_car_ids
        features = ModelData.item_features
        listings_item_embeddings = get_item_representations(features)[1]
        seen_item_embeddings = [listings_item_embeddings[latest_seen_car_ids - 1]]
        city_active_listings_embeddings = []
        for i in city_active_listings_ids:
            city_active_listings_embeddings.append(listings_item_embeddings[i - 1])
        max_cosine_scores = -np.ones(len(city_active_listings_ids)).reshape(1, -1)
        for item_embeddings in seen_item_embeddings:
            curr_cosine_scores = cosine_similarity(city_active_listings_embeddings, [item_embeddings]).reshape(1, -1)
            max_cosine_scores = np.maximum(curr_cosine_scores, max_cosine_scores)
        similarity_indices = np.argsort(-max_cosine_scores)
        similar_cars_cosine_scores = max_cosine_scores
        city_active_listings_ids = np.array(city_active_listings_ids)
        similar_listings = city_active_listings_ids[similarity_indices].ravel()
        listings.append(similar_listings[:4])
    f = pd.DataFrame(listings)
    f.columns = ["Listing ID", "Reco - 1", "Reco - 2", "Reco - 3"]
    f.to_csv('kkk.csv', encoding='utf-8', index=False)
    return f
        